In [28]:
# 2022-Dez-28
# Jupyter notebook for tests with OGRaySpY modules

In [29]:
import numpy as np
from numpy.random import default_rng
from numpy.polynomial import Polynomial as P
from scipy import special
import pandas as pd

from spec_class import Spec

In [30]:
# https://lmfit.github.io/lmfit-py/builtin_models.html
from lmfit import Model, minimize, Parameters, report_fit, fit_report, printfuncs
from lmfit.models import LinearModel, StepModel, ExponentialModel, LorentzianModel
from lmfit.models import GaussianModel, SkewedGaussianModel, ExpressionModel

In [31]:
# a spectrum pattern to find and test:

In [32]:
# 2022
# Se quiser ignorar o espectro aleatório, definir aqui um específico. Se não, pular.
# Vou ignorar o espectro já analisado e refazer a análise, para testes.
# 2022-Mar-24 Resolver essa questão dos nomes de arquivos em Linux e Windows
# a_spec_name = '..\\..\\Genie_Transfer\\Si\\2018\\PNR\\PRN0803.Chn'
# a_spec_name = '../../Genie_Transfer/Si/2018/PNR/PRN0803.Chn'
# a_spec_name = '../../Genie_Transfer/Si/2018/CTP/CTP0908-2.Chn'
# Esse é um que dá problema na baseline perto do 159 keV do I-123:
# a_spec_name = '../../Genie_Transfer/Si/2018/CCI/CCI1508I.Chn'
# Precisa quebrar mais a baseline: Ok, consertado!!!
# 2022: Já usando o GMX:
# a_spec_name = '../../Genie_Transfer/Filtros/2022/Cci/CCI1603-I.Chn'



In [33]:

# 2022-out-7: Excelente espectro para testes, tenho usado ultimamente:
# a_pattern = 'Si/SI2018/SI11318.Chn'
# 2022-nov-16: outros espectros:
# a_pattern = "Filtros/2022/Cci/CCI1603-I.Chn"
# a_pattern = "Filtros/2022/Cci/CCI2302-I.Chn"

# a_pattern = "Eso_non_existe.Chn"

# 2022-Dez-21 Buscando outro espectro
# a_pattern = 'CCI1603-I'
# 2022-Dez-28 Este é um espectro mais recente:
# a_pattern = 'SI09722.Chn'
# 2022-Dez-28 Este é um espectro mais recente que posso comparar com saída do Genie:
a_pattern = 'SI06322.Chn'

In [34]:
spectra_list_df = pd.read_pickle('WindowsI56611.pkl')
spectra_list_df

,info_plat,info_mach,info_syst,info_node,home_path,spectra_path,n_files,a_spec_ind,a_spec_name,reduced_f_name,gross_counts_graphics,pks_regions_gros,files_list,reduced_names_files_list,spectra_pattern_names,pkl_folder_files
0,Windows-10-10.0.19044-SP0,AMD64,Windows,I56611,C:\Users\mmaduar,C:\Users\mmaduar\Nextcloud\Genie_Transfer,6868,0,,,None,None,[C:\Users\mmaduar\Nextcloud\Genie_Transfer\alm...,"[almera22/det_c/iaeaqcb.chn, almera22/det_c/ia...","[**/*.[Cc][Hh][Nn], **/*.[Ii][Ee][Cc]]",WindowsI56611.pkl


In [35]:
def select_spectrum_from_folder_list(
        reduced_names_file_list, files_list, spectra_path, a_pattern='', random_spectrum=False):
    # Select a random spectrum...
    n_files = len(reduced_names_file_list)
    if random_spectrum:
        try:
            a_spec_ind = randrange(n_files)
            print('Random spec index: ', a_spec_ind)
            a_spec_name = files_list[a_spec_ind]
            reduced_f_name = reduced_names_file_list[a_spec_ind]
            print('...and its name: ', a_spec_name)
        except ValueError:
            print('No random spectrum found...')
        # ...or define it directly.
    else:
        print('Existing:')
        achou = False
        a_spec_ind = None
        nomearq = ''
        for i, j in enumerate(reduced_names_file_list):
            if a_pattern in j:
                achou = True
                a_spec_ind = i
                nomearq = j
                break
        if achou:
            print(f'Achou! indice={a_spec_ind}, nomearq = {nomearq}')
            a_spec_name = files_list[a_spec_ind]
            reduced_f_name = reduced_names_file_list[a_spec_ind]

        matching_spec_name = [i for i in spectra_path.glob(a_pattern)]
        if len(matching_spec_name) != 0:
            for i in matching_spec_name:
                print('name: ', i)
            a_spec_name = matching_spec_name[0]

    print('==========================')
    print('Final choices:')
    print(f'spectra_path: {spectra_path}')
    print(f'a_spec_name: {a_spec_name}')
    print(f'reduced_f_name: {reduced_f_name}')
    return a_spec_name, reduced_f_name

In [36]:
spectra_list_df = pd.read_pickle('WindowsI56611.pkl')
spectra_list_df

,info_plat,info_mach,info_syst,info_node,home_path,spectra_path,n_files,a_spec_ind,a_spec_name,reduced_f_name,gross_counts_graphics,pks_regions_gros,files_list,reduced_names_files_list,spectra_pattern_names,pkl_folder_files
0,Windows-10-10.0.19044-SP0,AMD64,Windows,I56611,C:\Users\mmaduar,C:\Users\mmaduar\Nextcloud\Genie_Transfer,6868,0,,,None,None,[C:\Users\mmaduar\Nextcloud\Genie_Transfer\alm...,"[almera22/det_c/iaeaqcb.chn, almera22/det_c/ia...","[**/*.[Cc][Hh][Nn], **/*.[Ii][Ee][Cc]]",WindowsI56611.pkl


In [37]:
reduc_nms = spectra_list_df.reduced_names_files_list[0]
fil_lst = spectra_list_df.files_list[0]
spc_pth = spectra_list_df.spectra_path[0]

In [38]:
a_spec_name, reduced_f_name = select_spectrum_from_folder_list(reduc_nms, fil_lst, spc_pth, a_pattern)

Existing:
Achou! indice=62, nomearq = almera22/user2023/SI2022/SI06322.Chn
Final choices:
spectra_path: C:\Users\mmaduar\Nextcloud\Genie_Transfer
a_spec_name: C:\Users\mmaduar\Nextcloud\Genie_Transfer\almera22\user2023\SI2022\SI06322.Chn
reduced_f_name: almera22/user2023/SI2022/SI06322.Chn


In [40]:
a_spec_name, reduced_f_name

(WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/almera22/user2023/SI2022/SI06322.Chn'),
 'almera22/user2023/SI2022/SI06322.Chn')

In [75]:
a_spec = Spec(a_spec_name, reduced_f_name)
a_spec.total_analysis(gener_dataframe=True)

Starting Spec.total_analysis...
Finish Spec.total_analysis!


In [76]:
vars(a_spec)

{'net_spec_ser_an': <generic_series_analysis_class.GenericSeriesAnalysis at 0x1335b3241c0>,
 'final_composed_baseline': <generic_series_analysis_class.GenericSeriesAnalysis at 0x1335b324640>,
 'f_name': WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/almera22/user2023/SI2022/SI06322.Chn'),
 'reduced_f_name': 'almera22/user2023/SI2022/SI06322.Chn',
 'sufx': '.chn',
 'spec_io': None,
 'lv_time': 50000.0,
 'rl_time': 50028.58,
 'source_datetime': None,
 'pkl_file': WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/almera22/user2023/SI2022/SI06322.pkl'),
 'gross_spec_ser_an': <generic_series_analysis_class.GenericSeriesAnalysis at 0x1335b324490>,
 'origin_spec_ser_an': <generic_series_analysis_class.GenericSeriesAnalysis at 0x1335b324580>,
 'start_datetime': datetime.datetime(2022, 8, 4, 19, 42, 23),
 'det_descr': 'MAQUINA_01 MCB 1 Input 1',
 'sam_descr': 'SI-063/22 - Agua da Piscina do Reator IEA-R1 (GMX)',
 'channel_energy_calib': <genericcalib_class.ChannelEnergyCalib at 0x1

In [77]:
vars(a_spec.net_spec_ser_an.pk_parms)

{'k_fwhm_to_sd': 0.42466090014400953,
 'peaks': array([  31,   42,  125,  128,  144,  235,  396,  488,  529,  582,  715,
         842,  960, 1003, 1082, 1115, 1130, 1162, 1196, 1257, 1333, 1372,
        1454, 1497, 1540, 1585, 1592, 1832, 1840, 1926, 2033, 2187, 2272,
        2398, 2470, 2614, 2837, 2895, 3030, 3450, 3615, 4014], dtype=int64),
 'pk_hei': array([], dtype=float64),
 'widths': (None, None),
 'mix_regions': array([], dtype=float64),
 'plateaux': array([], dtype=float64),
 'fwhm_plateaux': array([], dtype=float64),
 'wide_regions': array([[  27,   35],
        [  34,   54],
        [ 123,  127],
        [ 125,  131],
        [ 142,  147],
        [ 233,  238],
        [ 394,  398],
        [ 481,  494],
        [ 526,  532],
        [ 578,  585],
        [ 713,  717],
        [ 836,  847],
        [ 957,  964],
        [1000, 1006],
        [1078, 1086],
        [1110, 1120],
        [1126, 1133],
        [1158, 1167],
        [1194, 1198],
        [1251, 1261],
        [13

In [78]:
# 2022-Jun-24 No momento esses são os que interessam:
# pann = ogra.a_spec..saw_spec.spec_parms
cntarr = a_spec.net_spec_ser_an.y_s
pkprms = a_spec.net_spec_ser_an.pk_parms

In [79]:
vars(a_spec.channel_energy_calib)

{'calib_kind': 'channel energy',
 'p_en': Polynomial([-1.99049842,  0.61104035,  0.        ], domain=[-1,  1], window=[-1,  1]),
 'coeffs_ch_en': array([ 0.        ,  0.61104035, -1.99049842]),
 'en_ch_calib': array([], dtype=float64),
 'chan_calib': array([], dtype=float64)}

In [80]:
a_spec.channel_energy_calib.p_en(453.5)

275.1163020133972

In [81]:
a_spec.pkl_file

WindowsPath('C:/Users/mmaduar/Nextcloud/Genie_Transfer/almera22/user2023/SI2022/SI06322.pkl')

In [82]:
df_1 = pd.read_pickle(a_spec.pkl_file)
df_1

,net_spec_ser_an,final_composed_baseline,f_name,reduced_f_name,sufx,spec_io,lv_time,rl_time,source_datetime,pkl_file,gross_spec_ser_an,origin_spec_ser_an,start_datetime,det_descr,sam_descr,channel_energy_calib
0,<generic_series_analysis_class.GenericSeriesAn...,<generic_series_analysis_class.GenericSeriesAn...,C:\Users\mmaduar\Nextcloud\Genie_Transfer\alme...,almera22/user2023/SI2022/SI06322.Chn,.chn,None,50000.0,50028.58,None,C:\Users\mmaduar\Nextcloud\Genie_Transfer\alme...,<generic_series_analysis_class.GenericSeriesAn...,<generic_series_analysis_class.GenericSeriesAn...,2022-08-04 19:42:23,MAQUINA_01 MCB 1 Input 1,SI-063/22 - Agua da Piscina do Reator IEA-R1 (...,<genericcalib_class.ChannelEnergyCalib object ...


In [83]:
# 2023-Mar-20
# Lendo biblioteca de nuclídeos
pisciro_eng_df = pd.read_excel('../data/Pisciro-Eng_(2019).ods')
pisciro_eng_df

,Nucl,Engy,sEngy,Yie,sYie
0,Ag-110m,446.811,0.003,3.750,0.030
1,Ag-110m,620.360,0.003,2.806,0.018
2,Ag-110m,657.762,0.002,94.600,0.400
3,Ag-110m,677.723,0.002,10.350,0.080
4,Ag-110m,687.015,0.003,6.440,0.060
5,Ag-110m,706.682,0.003,16.440,0.100
6,Ag-110m,744.277,0.003,4.730,0.030
7,Ag-110m,763.944,0.003,22.290,0.090
8,Ag-110m,818.031,0.003,7.340,0.040
9,Ag-110m,884.685,0.003,72.700,0.300


In [84]:
vars_pkprms = vars(pkprms)
vars_pkprms

{'k_fwhm_to_sd': 0.42466090014400953,
 'peaks': array([  31,   42,  125,  128,  144,  235,  396,  488,  529,  582,  715,
         842,  960, 1003, 1082, 1115, 1130, 1162, 1196, 1257, 1333, 1372,
        1454, 1497, 1540, 1585, 1592, 1832, 1840, 1926, 2033, 2187, 2272,
        2398, 2470, 2614, 2837, 2895, 3030, 3450, 3615, 4014], dtype=int64),
 'pk_hei': array([], dtype=float64),
 'widths': (None, None),
 'mix_regions': array([], dtype=float64),
 'plateaux': array([], dtype=float64),
 'fwhm_plateaux': array([], dtype=float64),
 'wide_regions': array([[  27,   35],
        [  34,   54],
        [ 123,  127],
        [ 125,  131],
        [ 142,  147],
        [ 233,  238],
        [ 394,  398],
        [ 481,  494],
        [ 526,  532],
        [ 578,  585],
        [ 713,  717],
        [ 836,  847],
        [ 957,  964],
        [1000, 1006],
        [1078, 1086],
        [1110, 1120],
        [1126, 1133],
        [1158, 1167],
        [1194, 1198],
        [1251, 1261],
        [13

In [85]:
keys_to_get = ['fwhm_centr', 'rough_sums', 'centroids', 'variances']

In [86]:
prep_for_dict = [(key, vars_pkprms[key]) for key in keys_to_get]

In [87]:
pks_dict = dict(prep_for_dict)

In [88]:
peaks_df = pd.DataFrame.from_dict(pks_dict)

In [89]:
p_en = a_spec.channel_energy_calib.p_en
p_en

Polynomial([-1.99049842,  0.61104035,  0.        ], domain=[-1,  1], window=[-1,  1])

In [90]:
peaks_df['energy'] = p_en(peaks_df['centroids'])
peaks_df

,fwhm_centr,rough_sums,centroids,variances,energy
0,31.014996,1615.276915,31.185666,10253.822135,17.065202
1,43.801123,1420.007286,44.318418,21551.688710,25.089844
2,124.694854,1631.302448,125.249051,6686.697552,74.541726
3,128.281922,2934.532615,127.660103,10095.467385,76.014976
4,144.470020,1384.590198,144.548300,7785.409802,86.334346
5,235.390263,332.069350,235.142704,7906.952592,141.691183
6,396.019819,390.897453,396.262867,5348.525570,240.142104
7,487.815753,533.667297,488.146618,10969.661563,296.286784
8,529.359372,296.993919,529.164769,4995.071803,321.350529
9,581.614016,484.055638,581.772047,5194.312189,353.495699


In [91]:
# 2023-Mar-21: Identify nuclide candidates in peaks_df from pisciro_eng_df, with energy tolerance en_toler

In [92]:
big_merge = pd.merge(peaks_df, pisciro_eng_df, how='cross')
big_merge

,fwhm_centr,rough_sums,centroids,variances,energy,Nucl,Engy,sEngy,Yie,sYie
0,31.014996,1615.276915,31.185666,10253.822135,17.065202,Ag-110m,446.811,0.003,3.750,0.030
1,31.014996,1615.276915,31.185666,10253.822135,17.065202,Ag-110m,620.360,0.003,2.806,0.018
2,31.014996,1615.276915,31.185666,10253.822135,17.065202,Ag-110m,657.762,0.002,94.600,0.400
3,31.014996,1615.276915,31.185666,10253.822135,17.065202,Ag-110m,677.723,0.002,10.350,0.080
4,31.014996,1615.276915,31.185666,10253.822135,17.065202,Ag-110m,687.015,0.003,6.440,0.060
...,...,...,...,...,...,...,...,...,...,...
2053,4012.617584,175.718687,4011.784325,384.511576,2449.371615,W-187,479.550,0.022,21.800,0.800
2054,4012.617584,175.718687,4011.784325,384.511576,2449.371615,W-187,618.260,0.040,6.280,0.220
2055,4012.617584,175.718687,4011.784325,384.511576,2449.371615,W-187,685.730,0.040,27.300,1.000
2056,4012.617584,175.718687,4011.784325,384.511576,2449.371615,W-187,772.890,0.050,4.120,0.140


In [93]:
en_toler = 2.0

In [94]:
big_merge['delta_en'] = big_merge.energy - big_merge.Engy
big_merge

,fwhm_centr,rough_sums,centroids,variances,energy,Nucl,Engy,sEngy,Yie,sYie,delta_en
0,31.014996,1615.276915,31.185666,10253.822135,17.065202,Ag-110m,446.811,0.003,3.750,0.030,-429.745798
1,31.014996,1615.276915,31.185666,10253.822135,17.065202,Ag-110m,620.360,0.003,2.806,0.018,-603.294798
2,31.014996,1615.276915,31.185666,10253.822135,17.065202,Ag-110m,657.762,0.002,94.600,0.400,-640.696798
3,31.014996,1615.276915,31.185666,10253.822135,17.065202,Ag-110m,677.723,0.002,10.350,0.080,-660.657798
4,31.014996,1615.276915,31.185666,10253.822135,17.065202,Ag-110m,687.015,0.003,6.440,0.060,-669.949798
...,...,...,...,...,...,...,...,...,...,...,...
2053,4012.617584,175.718687,4011.784325,384.511576,2449.371615,W-187,479.550,0.022,21.800,0.800,1969.821615
2054,4012.617584,175.718687,4011.784325,384.511576,2449.371615,W-187,618.260,0.040,6.280,0.220,1831.111615
2055,4012.617584,175.718687,4011.784325,384.511576,2449.371615,W-187,685.730,0.040,27.300,1.000,1763.641615
2056,4012.617584,175.718687,4011.784325,384.511576,2449.371615,W-187,772.890,0.050,4.120,0.140,1676.481615


In [95]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#selection-by-callable
# MaTched ENergies
mt_en = pd.DataFrame(big_merge.loc[lambda df: abs(df['delta_en']) < en_toler, :]) \
[['Engy', 'Nucl', 'rough_sums', 'Yie']]
mt_en
# mt_en[['Engy', 'Nucl']]

,Engy,Nucl,rough_sums,Yie
413,320.084,Cr-51,296.993919,10.080
678,610.330,Ru-103,697.995119,5.730
688,657.762,Ag-110m,1959.419394,94.600
738,677.723,Ag-110m,288.714523,10.350
788,687.015,Ag-110m,218.053732,6.440
838,706.682,Ag-110m,353.212972,16.440
938,763.944,Ag-110m,435.252398,22.290
997,810.775,Co-58,2432.312117,99.448
1063,834.827,Mn-54,203.128675,99.975
1087,884.685,Ag-110m,1210.515798,72.700


In [96]:
# Now, we can insert the counting efficiency function:

In [97]:
# Curva de eficiência para frasco F100 do GMX (2021):
# Arquivo Effic_F100_GMX_2021.ods construído a partir de
# Nextcloud\Ipen\Radiometria\2021_Curvas-efic_Bessa\F100 GMX_2021.pdf

In [98]:
# 2023-Mar-21
# Lendo curva de eficiência
eff_F100_GMX_2021_df = pd.read_excel('../data/Effic_F100_GMX_2021.ods')
effics_df = eff_F100_GMX_2021_df
effics_df

,Engy,Effic,sEffic
0,59.54,6.4299,3.61
1,81.00,5.6528,3.61
2,276.40,1.6306,3.61
3,302.85,1.5006,3.61
4,356.02,1.4023,3.61
5,383.85,1.3216,3.61
6,604.69,0.9112,3.61
7,661.66,0.7900,3.61
8,795.84,0.7000,3.61
9,1155.00,0.5916,0.01


In [99]:
p_eff_log = P.fit(np.log(effics_df.Engy),
                  np.log(effics_df.Effic),
                  full=True,
                  w = 1/effics_df.sEffic, deg=5)
def p_eff(en):
    return np.exp(p_eff_log[0](np.log(en)))

In [100]:
p_eff_log

(Polynomial([ 0.51104917, -1.54282532,  0.42082601,  0.36310567, -0.3132035 ,
        -0.06636575], domain=[4.08664836, 7.19481216], window=[-1.,  1.]),
 [array([0.00174484]),
  6,
  array([2.44946008e+00, 1.04967799e-02, 5.50802223e-03, 2.01458065e-03,
         8.07092181e-04, 2.92462463e-04]),
  2.6645352591003757e-15])

In [101]:
p_eff(effics_df.Engy)

0     6.454368
1     5.607742
2     1.698686
3     1.552736
4     1.335262
5     1.249593
6     0.884475
7     0.834158
8     0.744488
9     0.591599
10    0.585366
11    0.533971
Name: Engy, dtype: float64

In [102]:
log_unc_fitted_effic = P([4.70033902423E+01, -2.82179877310E+01, 5.72952770082E+00, -3.88458849293E-01])
log_unc_fitted_effic

Polynomial([ 47.00339024, -28.21798773,   5.7295277 ,  -0.38845885], domain=[-1,  1], window=[-1,  1])

In [103]:
# Incerteza percentual da curva F100_GMX_2021 para os valores de eficiência
# Os coeficientes foram tirados do InterWinner e a curva reconstruída aqui.
unc_perc_fitted_effic = np.exp(log_unc_fitted_effic(np.log(effics_df.Engy)))
unc_perc_fitted_effic

0     2.366059
1     1.972769
2     1.533736
3     1.437744
4     1.233116
5     1.126118
6     0.470105
7     0.365696
8     0.198657
9     0.037309
10    0.034281
11    0.016454
Name: Engy, dtype: float64

In [104]:
# Sample size (L, kg, g etc.)
samp_size = 0.1

In [105]:
mt_en['uncorr_activ'] = mt_en['rough_sums'] /\
                        (p_eff(mt_en['Engy']) * 1e-4 * mt_en['Yie'] *
                         a_spec.lv_time * samp_size)

In [108]:
mt_en

,Engy,Nucl,rough_sums,Yie,uncorr_activ
413,320.084,Cr-51,296.993919,10.080,40.010732
678,610.330,Ru-103,697.995119,5.730,277.152231
688,657.762,Ag-110m,1959.419394,94.600,49.474571
738,677.723,Ag-110m,288.714523,10.350,67.906060
788,687.015,Ag-110m,218.053732,6.440,83.133178
838,706.682,Ag-110m,353.212972,16.440,53.687315
938,763.944,Ag-110m,435.252398,22.290,51.178552
997,810.775,Co-58,2432.312117,99.448,66.441829
1063,834.827,Mn-54,203.128675,99.975,5.616925
1087,884.685,Ag-110m,1210.515798,72.700,47.655927


In [ ]:
# 2023-Mar-22 PAREI AQUI

In [ ]:

msedg serg set asrhg saertrg

In [ ]:
rng = np.random.default_rng()
# rints = rng.integers(low=0, high=10, size=3)

# 2022-Jun-23: Reativar após consertar o arquivo pickle:
# rints = rng.integers(pkl_df1.count()[0], size=3)
# print(rints)
# type(rints[0])
# some_spec = rints[1]
# pkl_df1.iloc[some_spec]
# a_spec_name = pkl_df1.iloc[some_spec]['FName']
# a_spec_name

In [ ]:
# 2022-Dez-28: Atualizar:
# (2022-Jun-24 Lembrando como estão os defaults de total_analysis(...):
# def total_analysis(self, k_sep_pk=2.0, smoo=3000.0, widths_range=(4.0, 20.0)):)
# smoo=3000 fica razoável, mas faz umas ondas
# smoo=1500 fica horrível, faz uns abismos. Quanto menor, pior
# smoo=400 fica razoável, mas faz umas ondas
# ogra.perform_total_analysis(
#     k_sep_pk=2.0, smoo=4096, widths_range=(4.0, 20.0)
# )

In [107]:
# 2022-Dez-28
# Testes com conversão de pandas df .to_html
# Teste 1: html direto
df_to_html = a_spec.spec_pks_df.to_html(columns=['peaks', 'centroids', 'rough_sums'])
df_to_html

AttributeError: 'Spec' object has no attribute 'spec_pks_df'

In [ ]:
# 2022-Dez-28
# Testes com conversão de pandas df .to_html
# Teste 2: aplicando tags html:
df_to_html = ogra.a_spec.spec_pks_df.to_html(
    columns=['peaks', 'centroids', 'rough_sums'],
    formatters={
        'peaks': lambda x: '<b>' + str(x) + '</b>',
        'rough_sums': lambda x: '<i>' + str(x) + '</i>'}
)
df_to_html
import IPython
IPython.display.HTML(df_to_html)

In [ ]:
# 2022-Dez-28
# Testes com conversão de pandas df .to_html
# Teste 3: aplicando float_formatters:
df_to_html = ogra.a_spec.spec_pks_df.to_html(
    columns=['peaks', 'centroids', 'rough_sums'],
    float_format='{:.3f}'
)
df_to_html

In [ ]:
# Teste 3: usando Styles:
my_df = ogra.a_spec.spec_pks_df
my_df.style.format(na_rep='MISSING', thousands=" ",
                   formatter={('fwhm_centr', 'centroids'): "{:.2f}"})

In [ ]:
df = pd.DataFrame([[38.0, 2.0, 18.0, 22.0, 21, np.nan],[19, 439, 6, 452, 226,232]],
                  index=pd.Index(['Tumour (Positive)', 'Non-Tumour (Negative)'], name='Actual Label:'),
                  columns=pd.MultiIndex.from_product([['Decision Tree', 'Regression', 'Random'],['Tumour', 'Non-Tumour']], names=['Model:', 'Predicted:']))

df.style.format(precision=0, na_rep='MISSING', thousands=" ",
                formatter={('Decision Tree', 'Tumour'): "{:.2f}",
                           ('Regression', 'Non-Tumour'): lambda x: "$ {:,.1f}".format(x*-1e6)
                           })

In [ ]:
# 2022-Jun-14
# Verificando o que se tem até agora em a_raw_spec:
# (Pode pular e ir direto aos gráficos)

In [ ]:

# Testes:
# https://numpy.org/doc/stable/reference/random/generator.html#distributions
# x = np.linspace(0, 10, N) + rng.normal(loc=7, scale=0.05, size=N)
# y = np.linspace(22, 23, N)

# Busca do canal onde a altura do espectro se "estabiliza"
# https://numpy.org/doc/stable/reference/routines.sort.html#searching



In [ ]:
vars(pkprms)

In [ ]:
# 2022-Dez-21 Achar o pico de maior área inicial do espectro:

In [ ]:
largest_pk = np.argmax(pkprms.rough_sums)
pkprms.centroids[largest_pk]

In [ ]:
# Só pra lembrar como faz divisão inteira
21 // 7

In [ ]:
wi_re = pkprms.wide_regions[largest_pk]
x = np.linspace(wi_re[0], wi_re[1], num=wi_re[1]-wi_re[0]+1)
y = cntarr[wi_re[0]:wi_re[1]+1]
c1 =  pkprms.centroids[largest_pk]
s1 = pkprms.propts['widths'][largest_pk]*pkprms.k_fwhm_to_sd
a1 = pkprms.rough_sums[largest_pk]
# c2 = 313
# s2 = 1.5
# a2 = 400
sste = 2.0

In [ ]:
# 2023-Jan-3
# Aqui finalizo este teste:
# Dado um espectro, escolhi o pico mais proeminente do espectro
# e fiz o ajuste gaussianco com lmfit
# No caso, o pico tem área, centroide, SD e FWHM:
# g1_amplitude:  5264.84728 +/- 85.1465566 (1.62%) (init = 5347.497)
# g1_center:     1926.29564 +/- 0.02690572 (0.00%) (init = 1926.275)
# g1_sigma:      1.44040103 +/- 0.02690060 (1.87%) (init = 1.418733)
# g1_fwhm:       3.39188516 +/- 0.06334608 (1.87%) == '2.3548200*g1_sigma'
# g1_height:     1458.18438 +/- 23.5824677 (1.62%) == '0.3989423*g1_amplitude/max(1e-15, g1_sigma)'

# Comparando com saíuda do Genie:
# canal (>0)    En:    FWHM (keV)    net area    net area unc   continuum
# 1927.34      1173.47 2.04          5.40E+003   89.73          8.83E+002
#

In [ ]:
out

In [ ]:
out.params

In [ ]:
out.var_names

In [ ]:
out.params.items()

In [ ]:
out.params.pretty_print()

In [ ]:
print('-------------------------------')
print('Parameter    Value       Stderr')
for name, param in out.params.items():
    # print(f'{name:7s} {param.value:11.5f} {param.stderr:11.5f}')
    print(name)
    print(param)
    print(param.value)

In [ ]:
def gauss_lmfit_singlet(pkprms, ipk):
    wi_re = pkprms.wide_regions[ipk]
    print(f'pico: {wi_re}')
    x = np.linspace(wi_re[0], wi_re[1], num=wi_re[1]-wi_re[0]+1)
    y = cntarr[wi_re[0]:wi_re[1]+1]
    c1 = pkprms.centroids[ipk]
    s1 = pkprms.propts['widths'][ipk]*pkprms.k_fwhm_to_sd
    a1 = pkprms.rough_sums[ipk]

    parames = Parameters()

    gauss1 = GaussianModel(prefix='g1_')
    parames.update(gauss1.make_params())
    parames['g1_center'].set(value=c1, min=c1-2, max=c1+2)
    parames['g1_sigma'].set(value=s1, min=0.5, max=10)
    parames['g1_amplitude'].set(value=a1, min=10)

    mod = gauss1

    init = mod.eval(parames, x=x)
    out = mod.fit(y, parames, x=x, verbose=False)
    print('Fez.')

    # ret = [param for param in out.params.items()]
    ret = out
    return ret

In [ ]:
for ipk in range(len(pkprms.peaks)):
    ret = gauss_lmfit_singlet(pkprms, ipk)
    print(ret)

In [ ]:
ret

In [ ]:
ret.params

In [ ]:
for name, param in ret.params.items():
    print(name, param.value, param.stderr)

In [ ]:
for ipk in range(len(pkprms.peaks)):
    ret = gauss_lmfit_singlet(pkprms, ipk)
    # ret.params.pretty_print()

In [ ]:
ret

In [ ]:
xfgn sr hsrthf srth

In [ ]:
my_df.dtypes

In [ ]:
ch_en = ogra.a_spec.channel_energy_calib.p_en

In [ ]:
my_df['centrs_en'] = ch_en(my_df['centroids'])
my_df

In [ ]:

# 2022-Dez-21:
# PAREI AQUI;
# Usar os resultados de áreas dos picos e montar as curvas de eficiência

# 2022-Mar-16
# Fazendo ajuste polinomial nas eficiências:

In [ ]:
from numpy.polynomial import Polynomial as P # 2020-09-06 Esta é a nova classe recomendada 

In [ ]:
En_CV_LoBg = np.array([81.00,
86.54,
121.78,
133.51,
302.85,
344.28,
356.02,
604.69,
661.66,
795.84,
964.13,
1085.91,
1112.12,
1173.24,
1332.50,
1408.01
])
# En_CV_GMX

In [ ]:
Eff_CV_LoBg = np.array([4.4358,
4.4723,
5.4386,
5.5045,
3.1379,
2.8722,
2.7305,
1.7817,
1.6453,
1.4026,
1.1216,
1.0200,
0.9703,
0.8962,
0.8009,
0.7397 ])
Eff_CV_GMX

In [ ]:
unc_perc = np.array([2.01,
1.43,
1.33,
2.11,
1.81,
1.42,
1.41,
2.24,
1.44,
3.68,
1.66,
1.71,
1.58,
0.87,
0.87,
1.51 ])
unc_perc

In [ ]:
ln_En_CV_LoBg = np.log (En_CV_LoBg)
ln_Eff_CV_LoBg = np.log (Eff_CV_LoBg)

In [ ]:
N = 16

In [ ]:
# 2022-Mar-23
# Parêntese para fazer nova calibração canal-energia do HPGe do projeto:

In [ ]:
canal = np.array([210, 920, 1181, 7232])
ener = np.array([46.5, 186, 238.6, 1460.8])

In [ ]:
projeto = P.fit(x=canal, y=ener, deg=1).convert()
projeto

In [ ]:
projeto_ener_canal = P.fit(x=ener, y=canal, deg=1).convert()
projeto_ener_canal

In [ ]:
projeto_ener_canal(1173.2)

In [ ]:
projeto_ener_canal(1332.486)

In [ ]:
P.fit(x=ln_En_CV_LoBg, y=ln_Eff_CV_LoBg, deg=5).convert()

In [ ]:
poly_w = P.fit(x=ln_En_CV_LoBg, y=ln_Eff_CV_LoBg, deg=5, w=1/unc_perc).convert()
poly_w

In [ ]:
poly_w = P.fit(x=ln_En_CV_LoBg, y=ln_Eff_CV_LoBg, deg=5, w=1/unc_perc**2).convert()
poly_w

In [ ]:
print(poly_w)

In [ ]:
poly_o = P.fit(x=ln_En_CV_LoBg, y=Eff_CV_LoBg, deg=5)
print(poly_o)

In [ ]:
weights = np.linspace(1, 2, N)
Xw = X * np.sqrt(weights)[:, None]
yw = y * np.sqrt(weights)
print (linalg.lstsq(Xw, yw)[0])

In [ ]:
weights = 1.0 / unc_perc**2.0
weights

In [ ]:
Xw = ln_En_CV_LoBg * np.sqrt(weights)[:, None]
yw = ln_Eff_CV_LoBg * np.sqrt(weights)
print (linalg.lstsq(Xw, yw)[0])

In [ ]:
print (linalg.lstsq(Xw, yw))

In [ ]:
X = ln_En_CV_LoBg
y = ln_Eff_CV_LoBg
w1 = linalg.lstsq(X, y)
print (w1)

In [ ]:
np.log(2)

In [ ]:
# 2022-Dez-28
# Cópia do programa principal com interface, ograyspy_main_ui.py,
# para testes com Pandas

In [ ]:
import sys
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWebEngineWidgets import QWebEngineView
from PyQt5.QtWidgets import *
from css_test import css_test, apply_css

from ograyspy_class import Ograyspy
# from html_window_class import SimpleHtmlViewer

class MainWindow(QMainWindow):
    count = 0

    def __init__(self, parent=None):
        super(MainWindow, self).__init__(parent)
        self.mdi = QMdiArea()

        self.mdi.setHorizontalScrollBarPolicy(Qt.ScrollBarAsNeeded)
        self.mdi.setVerticalScrollBarPolicy(Qt.ScrollBarAsNeeded)
        self.setCentralWidget(self.mdi)

        # self.mdi.subWindowActivated.connect(self.updateMenus)
        self.windowMapper = QSignalMapper(self)
        # self.windowMapper.mapped[QWidget].connect(self.setActiveSubWindow)

        self.setCentralWidget(self.mdi)
        bar = self.menuBar()

        file = bar.addMenu("File")
        file.addAction("New")
        file.addAction("Open")
        file.addAction("Pandas-generate an_html_file.html")
        file.addAction("Show spectrum graphic")
        file.addAction("Show Pandas dataframe")
        file.addAction("cascade")
        file.addAction("Tiled")
        file.addAction("Exit")
        file.triggered[QAction].connect(self.windowaction)
        self.setWindowTitle("MDI demo")

    def windowaction(self, q):
        print("triggered")

        if q.text() == "New":
            MainWindow.count = MainWindow.count + 1
            sub = QMdiSubWindow()
            sub.setWidget(QTextEdit())
            sub.setWindowTitle("subwindow" + str(MainWindow.count))
            self.mdi.addSubWindow(sub)
            sub.show()

        if q.text() == "Open":
            fileName, _ = QFileDialog.getOpenFileName(self)
            if fileName:
                # This is a hack...
                # existing = self.findMdiChild(fileName)
                existing = True
                if existing:
                    MainWindow.count = MainWindow.count + 1
                    sub = QMdiSubWindow()
                    sub.setWidget(QTextEdit())
                    # sub.setWindowTitle("subwindow" + str(MainWindow.count))
                    sub.setWindowTitle(fileName)
                    self.mdi.addSubWindow(sub)
                    sub.show()

        if q.text() == "Pandas-generate an_html_file.html":
            fileName, _ = QFileDialog.getOpenFileName(self)
            if fileName:
                # This is a hack...
                # existing = self.findMdiChild(fileName)
                existing = True
                if existing:
                    ogra = Ograyspy(batch_mode=False)
                    # to_be_found = 'Genie_Transfer'
                    # print('\nogra.define_files_folder(to_be_found)')
                    # ogra.define_files_folder(to_be_found)
                    ogra.a_spec_name = fileName
                    # AQUI: ativar gener_dataframe qdo estiver pronto.
                    ogra.perform_total_analysis(peak_sd_fact=3.0, gener_dataframe=True)
                    ogra.a_spec.spec_pks_df.to_html(buf='an_html_file.html')
                    # print(ogra.a_spec.spec_pks_df)

                    MainWindow.count = MainWindow.count + 1
                    sub = QMdiSubWindow()
                    web = QWebEngineView()
                    # web.load(QUrl("file:///C:/Users/mmaduar/PycharmProjects/OGRaySpY/src/my_file.html"))
                    apply_css(ogra.a_spec.spec_pks_df)
                    web.load(QUrl("file:///C:/Users/mmaduar/PycharmProjects/OGRaySpY/src/an_html_file.html"))
                    # 2022-Dez-27 PAREI AQUI:
                    # - passar a saída de to_html como string?
                    # - O html pode ser renderizado com sucesso, inclusive com css
                    # - porém, parece que o SEI não aceita css. Paciência.

                    # web.load(ogra.dataframe_html_string)
                    # css_test()
                    sub.setWidget(web)
                    sub.setWindowTitle("subwindow" + str(MainWindow.count))
                    self.mdi.addSubWindow(sub)
                    sub.show()

        if q.text() == "Show spectrum graphic":
            MainWindow.count = MainWindow.count + 1
            sub = QMdiSubWindow()
            web = QWebEngineView()
            web.load(QUrl("file:///C:/Users/mmaduar/PycharmProjects/OGRaySpY/src/gross_counts_graph.html"))
            sub.setWidget(web)
            sub.setWindowTitle("subwindow" + str(MainWindow.count))
            self.mdi.addSubWindow(sub)
            sub.show()

        if q.text() == "Show Pandas dataframe":
            MainWindow.count = MainWindow.count + 1
            sub = QMdiSubWindow()
            web = QWebEngineView()
            web.load(QUrl("file:///C:/Users/mmaduar/PycharmProjects/OGRaySpY/src/my_file.html"))
            # 2022-Dez-23 PAREI AQUI - passar a saída de to_html como string
            # web.load(ogra.dataframe_html_string)
            css_test()
            sub.setWidget(web)
            sub.setWindowTitle("subwindow" + str(MainWindow.count))
            self.mdi.addSubWindow(sub)
            sub.show()

        if q.text() == "cascade":
            self.mdi.cascadeSubWindows()

        if q.text() == "Tiled":
            self.mdi.tileSubWindows()

        if q.text() == "Exit":
            QApplication.instance().exit()

    def findMdiChild(self, fileName):
        canonicalFilePath = QFileInfo(fileName).canonicalFilePath()

        for window in self.mdiArea.subWindowList():
            if window.widget().currentFile() == canonicalFilePath:
                return window
        return None


def main():
    app = QApplication(sys.argv)
    ex = MainWindow()
    ex.show()
    sys.exit(app.exec_())


if __name__ == '__main__':
    main()


In [ ]:
dir()